In [ ]:
import os
import pandas as pd
import panel as pn
import numpy as np
import datetime as dt
import holoviews as hv
import datetime
from colorcet import fire
from holoviews.operation.datashader import rasterize
from sathelpers import SatelliteDataStore
hv.extension('bokeh')

In [ ]:
# Set some configuration variables
# In general, these should be explicit paths with no variables or homedir (~)
AIS_DIR = "/Users/pwang/data/vault"
SAT_DIR = "/Users/pwang/data/vault/satellites_active"

if not os.path.isdir(AIS_DIR) or not os.path.isdir(SAT_DIR):
    raise IOError("Invalid source data directory")

### Get metadata

Next line is not cross-platform:

In [ ]:
!wget https://www.ucsusa.org/sites/default/files/2020-10/UCS-Satellite-Database-8-1-2020.txt

In [ ]:
df = pd.read_csv("UCS-Satellite-Database-8-1-2020.txt", sep='\t', encoding='L1', low_memory=False) 
df = df.dropna(axis='columns',how='all')
df.head(3)

In [ ]:
norad_names = dict(zip(df['Name of Satellite, Alternate Names'], df['NORAD Number']))
norad_names.pop([el for el in list(norad_names.keys()) if type(el) != str][0]) # Drop nan record

### Define selector input

In [ ]:
selector = pn.widgets.AutocompleteInput(
    name='Satellite', options=list(norad_names.keys()),
    placeholder='Satellite name')
selector.value = 'International Space Station (ISS [first element Zarya])'
selector

## Step 0. Configure the input parameters

In [ ]:
# Based on the year of interest, also define the AIS file to look at
AIS_FILENAME = "ais_2015.h5"

## Step 1. Load the satellite data

In [ ]:
satdata = SatelliteDataStore(SAT_DIR)

## Step 2. Load the AIS data

Since the example in this notebook is from the period of time of 2009, we just need to load its AIS tracks.

In [ ]:
ais = pd.read_hdf(os.path.join(AIS_DIR, AIS_FILENAME))
ais.sort_values(by="date_time", inplace=True)
ais.info()

## Step 3. Compute the visible points

In [ ]:
import sys
sys.path.insert(0, "../vault/Scripts")   # TODO
import intersect; intersect.PRINT_INFO=False

## Step 4. Visualize the results

Start by defining date pickers:

In [ ]:
start_date = pn.widgets.DatePicker(name='Start Date', value=datetime.date(2015, 1, 1))
end_date = pn.widgets.DatePicker(name='End Date', value=datetime.date(2015, 1, 4))

In [ ]:

tiles = hv.element.tiles.ESRI().redim(x='easting', y='northing')

def modulo_lon(val):
    return (val+180) % 360 - 180

def get_track(lat, lon, lat_clip=85.5):
    #lat, lon = track[1,:], track[2,:]
    mask = np.abs(lat) > lat_clip
    lat[mask] = np.float('nan')
    lon[mask] = np.float('nan')
    lon = np.array([modulo_lon(el) for el in lon])
    
    eastings, northings = hv.util.transform.lon_lat_to_easting_northing(lon,lat)
    # Heuristic to insert NaNs to break up Curve (prevent wrapping issues at date line)
    inds = np.where(np.abs(np.diff(eastings)) > 2e7)[0] # Big delta to split on
    inds += 1
    eastings  = np.insert(eastings,  inds, [float('nan') for i in range(len(inds))])
    northings = np.insert(northings, inds, [float('nan') for i in range(len(inds))])
    return hv.Curve((eastings, northings))

def rasterize_hits(name_dict, start_dict, end_dict, plot_size_dict, rangexy_dict):#
    name, start_date, end_date = name_dict['value'], start_dict['value'], end_dict['value']
    norad_id = int(norad_names[name])
    start_time = pd.Timestamp(start_date)
    end_time = pd.Timestamp(end_date)
    try:
        (times, lats, lons, alts) = satdata.get_precomputed_tracks(norad_id,
                                                            start=start_time, end=end_time)
    except:
        return hv.Curve(None)
    mask = lons > 180.0
    lons[mask] -= 360  # Need longitudes in (-180,180) format, not 0-360
    
    sat = pd.DataFrame({"date_time": times.astype("<M8[s]"),"lat": lats, "lon": lons, "alt": alts})
    hits = intersect.compute_hits(sat, ais, 
                                  start_time=str(start_date), 
                                  end_time=str(end_date), workers=4)
    mask = (np.abs(hits['lat']) < 85)
    eastings, northings = hv.util.transform.lon_lat_to_easting_northing(hits['lon'], hits['lat'])
    
    return rasterize(hv.Points(pd.DataFrame({'northing':northings[mask], 
                    'easting':eastings[mask]}), ['easting', 'northing']),
                             width = int(plot_size_dict['width']),
                             height = int(plot_size_dict['height']),
                             x_range=rangexy_dict['x_range'],
                             y_range=rangexy_dict['y_range'],
                             dynamic=False
                            ).opts(cmap=fire[180:], width=700, height=500,
            cnorm='eq_hist', alpha=0.5) * get_track(lats, lons).opts(color='red')

In [ ]:
hits_dmap = hv.DynamicMap(rasterize_hits, 
                          streams=[selector.param.value,  start_date.param.value, end_date.param.value,
                                    hv.streams.PlotSize(width=700, height=500),  hv.streams.RangeXY()],
                     positional_stream_args=True)

In [ ]:
import warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)

In [ ]:
pn.Column(pn.Row(start_date, end_date), selector, tiles * hits_dmap)